<a href="https://colab.research.google.com/github/kusuma2506/AI-AGENT/blob/main/movie_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np

# Load IMDb dataset word index (to simulate tokenizer)
word_index = imdb.get_word_index()
index_word = {v+3: k for k, v in word_index.items()}
word_index = {k: (v+3) for k, v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2
word_index["<UNUSED>"] = 3

# Load pretrained model (or build one if needed)
# For demo, we'll build a quick one below. You can skip this block if you have a model already.

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense

# Load dataset
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=10000)
x_train = pad_sequences(x_train, maxlen=256, padding='post')
x_test = pad_sequences(x_test, maxlen=256, padding='post')

# Build and train a simple model (train once and save)
model = Sequential([
    Embedding(10000, 16),
    GlobalAveragePooling1D(),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=2, batch_size=512, validation_split=0.2)
model.save("movie_review_model.h5")

# Load model (use this part once saved)
model = load_model("movie_review_model.h5")

# Function to convert review text to sequence
def review_to_sequence(text):
    tokens = text.lower().split()
    seq = [word_index.get(word, 2) for word in tokens]
    return pad_sequences([seq], maxlen=256, padding='post')

# Prediction function
def predict_review(text):
    sequence = review_to_sequence(text)
    prediction = model.predict(sequence)[0][0]
    return "Hit 🎬" if prediction > 0.5 else "Flop 💣"

# Main
if __name__ == "__main__":
    review = input("Enter a movie review: ")
    result = predict_review(review)
    print("Prediction:", result)

Epoch 1/2
40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - accuracy: 0.5295 - loss: 0.6917 - val_accuracy: 0.7354 - val_loss: 0.6838
Epoch 2/2
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.7143 - loss: 0.6796 - val_accuracy: 0.7500 - val_loss: 0.6590


Enter a movie review: good
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
Prediction: Hit 🎬
